In [1]:
from data.data import Data
from model.model import *

data_raw = Data().get_all_data()

df2 = data_raw['MBTI 500']
df2 = df2[['type', 'posts']]
df2.rename(columns={'posts': 'text'}, inplace=True)
df2 = df2.sample(n = 1000, ignore_index = True, random_state = 1)

In [2]:
from scripts.Preprocessing_full import training_preprocessing, training_oversampling, training_balancing, training_vectorize


df2a = training_preprocessing(df2)
df3 = training_oversampling(df2a)
df4 = training_balancing(df3)
df_list = training_vectorize(df4)


Dataset contains 122553 rows

cleaned dataset contains 1000 rows and 8 columns
E-I dataset contains 348 rows and 2461 columns
S-N dataset contains 114 rows and 2665 columns
F-T dataset contains 579 rows and 2439 columns
P-J dataset contains 818 rows and 2430 columns


In [3]:
from model.model import *
model = initialize_model()

hist = train_model(df_list, model)

hist

/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/

F1-score:49.977
Model Type: type


/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversi

F1-score:46.154
Model Type: type


/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/

F1-score:54.05
Model Type: type


/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/aforbesj/.pyenv/versions/3.10.6/envs/post2personality/lib/

F1-score:48.842999999999996
Model Type: type


([SGDClassifier(), SGDClassifier(), SGDClassifier(), SGDClassifier()],
 [{'e': {'precision': 0.3076923076923077,
    'recall': 0.25,
    'f1-score': 0.27586206896551724,
    'support': 32},
   'i': {'precision': 0.6962025316455697,
    'recall': 0.7534246575342466,
    'f1-score': 0.7236842105263158,
    'support': 73},
   'accuracy': 0.6,
   'macro avg': {'precision': 0.5019474196689386,
    'recall': 0.5017123287671232,
    'f1-score': 0.49977313974591653,
    'support': 105},
   'weighted avg': {'precision': 0.5777994157740993,
    'recall': 0.6,
    'f1-score': 0.5872050816696915,
    'support': 105},
   'bal_acc': 0.5019474196689386},
  {'n': {'precision': 0.8571428571428571,
    'recall': 1.0,
    'f1-score': 0.923076923076923,
    'support': 30},
   's': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 5},
   'accuracy': 0.8571428571428571,
   'macro avg': {'precision': 0.42857142857142855,
    'recall': 0.5,
    'f1-score': 0.4615384615384615,
    'support': 35},
 